# Sentiment Analysis

## Motivation

I am a huge movie buff. My favourite thing to do is to watch movies. And I wanted to try a sentiment analysis in Python. Therefore I thought it would be great to do a sentiment analysis on the IMDb and Rotten Tomatoes datasets. Selfishly to generate an analysis for me!

# Data

- [The Open Movie Database](http://www.omdbapi.com/) <br />
A RESTful API database to access movie information. Last updated 2019. <br />
- [Rotten Tomatoes](https://www.kaggle.com/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset) <br />
Good old Kaggle dataset from Rotten Tomatoes. Note for future self - to access Rotten Tomatoes API you need to register for a key and they will not give this out for general use. <br />
- [IMDb Datasets](https://www.imdb.com/interfaces/)<br />
Direct from the website or from here:
- [IMDb Dataset 2](https://aws.amazon.com/marketplace/pp/prodview-rvhjlbc3larps?sr=0-1&ref_=beagle&applicationId=AWSMPContessa#usage)<br />
AWS IMDb dataset sample